In [ ]:
#penalized cox model to predict factors influencing donors time to return

# Cox Penalized Regression model with LASSO Penalty

#-- two models after completed and after deferral
#keep only WB 

import pandas as pd
import numpy as np
import datetime as dt
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

from sksurv.linear_model import CoxnetSurvivalAnalysis
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import FitFailedWarning
import sksurv.util
import warnings

df=pd.read_csv("../1_data/private/preprocessed_data.csv")

mask = (df['Visit_yr'] >= 2017) 
df = df.loc[mask]

In [ ]:
print(list(df.columns))

In [ ]:
# convert columns to categorical varibles
cols = ['SEX', 'SPONSORTYPE', 'ABO_RH', 'DONTYPE', 'TRANSFUS', 'BORNUSA', 'EDUCATION', 'FTEVER', 'PREGNANT', 'NUMPREG', 'ETHNICITY', 'RACERECODE', 'THIRTYDSMOKED' ]
df_cox[cols] = df_cox[cols].astype('category')

df_cox = pd.get_dummies(df_cox, prefix=cols, dummy_na=True, columns = cols)

covariates = ['AGE', 'DAYS_SINCE_LAST_VISIT', 'DAYS_UNTIL_NEXT_VISIT', 'if_repeat', 'donor_status', 'next_if_defer', 'outcome_next', 'RBCunits.x', 'RBCunits.y', 'RBC_units_last_18m', 'RBCunits', 'HB_def_last_18m', 'HB_mean_last_18m', 'HB_min_last_18m', 'NumDonations_last_18m', 'NumVists_last_18m', 'SEX_0.0', 'SEX_1.0', 'SEX_9.0', 'SEX_nan', 'SPONSORTYPE_999', 'SPONSORTYPE_CHU', 'SPONSORTYPE_CIV', 'SPONSORTYPE_COL', 'SPONSORTYPE_COR', 'SPONSORTYPE_FIX', 'SPONSORTYPE_HEA', 'SPONSORTYPE_HS', 'SPONSORTYPE_IND', 'SPONSORTYPE_MIL', 'SPONSORTYPE_SER', 'SPONSORTYPE_nan', 'ABO_RH_A+', 'ABO_RH_A-', 'ABO_RH_AB+', 'ABO_RH_AB-', 'ABO_RH_B+', 'ABO_RH_B-', 'ABO_RH_O+', 'ABO_RH_O-', 'ABO_RH_OH+', 'ABO_RH_nan', 'DONTYPE_9', 'DONTYPE_A', 'DONTYPE_D', 'DONTYPE_H', 'DONTYPE_O', 'DONTYPE_R', 'DONTYPE_T', 'DONTYPE_nan', 'TRANSFUS_9', 'TRANSFUS_N', 'TRANSFUS_Y', 'TRANSFUS_nan', 'BORNUSA_-1.0', 'BORNUSA_0.0', 'BORNUSA_1.0', 'BORNUSA_nan', 'EDUCATION_A', 'EDUCATION_B', 'EDUCATION_C', 'EDUCATION_D', 'EDUCATION_E', 'EDUCATION_F', 'EDUCATION_S', 'EDUCATION_nan', 'FTEVER_-1.0', 'FTEVER_0.0', 'FTEVER_1.0', 'FTEVER_nan', 'PREGNANT_-1.0', 'PREGNANT_0.0', 'PREGNANT_1.0', 'PREGNANT_nan', 'NUMPREG_0.0', 'NUMPREG_1.0', 'NUMPREG_2.0', 'NUMPREG_3.0', 'NUMPREG_4.0', 'NUMPREG_5.0', 'NUMPREG_6.0', 'NUMPREG_nan', 'ETHNICITY_-1.0', 'ETHNICITY_0.0', 'ETHNICITY_1.0', 'ETHNICITY_nan', 'RACERECODE_1', 'RACERECODE_2', 'RACERECODE_3', 'RACERECODE_4', 'RACERECODE_5', 'RACERECODE_88', 'RACERECODE_O', 'RACERECODE_nan', 'THIRTYDSMOKED_9', 'THIRTYDSMOKED_N', 'THIRTYDSMOKED_Y', 'THIRTYDSMOKED_nan']
#- so deferral period is a covariate
#- deferral is a covariate
#- index donation is a covarite (M,F)
#- split by repeat, first time and recativated
#- add one one column for idi


In [ ]:
#helper functions to make plots

#a function to plot coefficients 
def plot_coef(coefs, n_highlight):
    _, ax = plt.subplots(figsize=(9, 6))
    n_features = coefs.shape[0]
    alphas = coefs.columns
    for row in coefs.itertuples():
        ax.semilogx(alphas, row[1:], ".-", label=row.Index)
        
    alpha_min = alphas.min()
    top_coefs = coefs.loc[:, alpha_min].map(abs).sort_values().tail(n_highlight)
    for name in top_coefs.index:
        coef = coefs.loc[name, alpha_min]
        plt.text(
            alpha_min, coef, name + "   ",
            horizontalalignment="right",
            verticalalignment="center"
        )

    ax.yaxis.set_label_position("right")
    ax.yaxis.tick_right()
    ax.grid(True)
    ax.set_xlabel("alpha")
    ax.set_ylabel("coefficient")
    
#plot mean concordance index and std deviation for each alpa across all folds
def plot_mean_concordance_index(alphas, mean, std, gridCV):
    fig, ax = plt.subplots(figsize=(9, 6))
    ax.plot(alphas, mean)
    ax.fill_between(alphas, mean - std, mean + std, alpha=.15)
    ax.set_xscale("log")
    ax.set_ylabel("concordance index")
    ax.set_xlabel("alpha")
    ax.axvline(gridCV.best_params_["coxnetsurvivalanalysis__alphas"][0], c="C1")
    ax.axhline(0.5, color="grey", linestyle="--")
    ax.grid(True)
    
#plot coefficients to be used
def plot_coef_to_use(non_zero_coefs, coef_order):
    _, ax = plt.subplots(figsize=(6, 8))
    non_zero_coefs.loc[coef_order].plot.barh(ax=ax, legend=False)
    ax.set_xlabel("coefficient")
    ax.grid(True)

In [ ]:
#helper function for feature selection

def feature_selection(cox_lasso, X_train, Y_train):
    
    #generate and plot coeffecients to choose best aplha
    coef_lasso = pd.DataFrame(
        cox_lasso.coef_,
        index=X_train.columns,
        columns=np.round(cox_lasso.alphas_, 5)
    )

    plot_coef(coef_lasso, n_highlight=5)
    
    #choosing the right aplha
    pipe = make_pipeline(
        StandardScaler(),
        CoxnetSurvivalAnalysis(l1_ratio=0.9, alpha_min_ratio=0.01, max_iter=100)
    )
    warnings.simplefilter("ignore", UserWarning)
    warnings.simplefilter("ignore", FitFailedWarning)
    pipe.fit(X_train, Y_train)
    
    #performing 5 fold cross validation
    estimated_alphas = pipe.named_steps["coxnetsurvivalanalysis"].alphas_
    cv = KFold(n_splits=5, shuffle=True, random_state=0)
    gridCV = GridSearchCV(
            make_pipeline(StandardScaler(), CoxnetSurvivalAnalysis(l1_ratio=0.9)),
            param_grid={"coxnetsurvivalanalysis__alphas": [[v] for v in estimated_alphas]},
            cv=cv,
            error_score=0.5,
            n_jobs=4).fit(X_train, Y_train)

    results = pd.DataFrame(gridCV.cv_results_)
    
    #plot mean_concordance index
    alphas = results.param_coxnetsurvivalanalysis__alphas.map(lambda x: x[0])
    mean = results.mean_test_score
    std = results.std_test_score
    
    plot_mean_concordance_index(alphas, mean, std, gridCV)
    
    #choose best model, 
    best_model = gridCV.best_estimator_.named_steps["coxnetsurvivalanalysis"]
    best_coefs = pd.DataFrame(
        best_model.coef_,
        index=X_train.columns,
        columns=["coefficient"]
    )

    #number of coefficients that are not zero
    non_zero = np.sum(best_coefs.iloc[:, 0] != 0)
    print("Number of non-zero coefficients: {}".format(non_zero))

    
    #create a list of coefs to be used
    non_zero_coefs = best_coefs.query("coefficient != 0")
    coef_order = non_zero_coefs.abs().sort_values("coefficient").index
    
    #plot_coefficients to be used
    plot_coef_to_use(non_zero_coefs, coef_order)
    
    return gridCV


In [ ]:
#model fitting
df_cox_def = df_cox.query("DEF_TYPE == 'low hgb'")
X = df_cox_def.loc[:, covariates]

   #Structured array
Y= sksurv.util.Surv.from_dataframe('CENSORED', 'time_to_event', df_cox_def)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.90, random_state=42)

#fit the model
cox_lasso = CoxnetSurvivalAnalysis(l1_ratio=1.0, alpha_min_ratio=0.01)
cox_lasso.fit(X_train, Y_train)

gcv = feature_selection(cox_lasso, X_train, Y_train)


In [ ]:
#prediction

#fit test set to perfrom prediction of hazard function and survival function
coxnet_pred = make_pipeline(
    StandardScaler(),
    CoxnetSurvivalAnalysis(l1_ratio=0.9, fit_baseline_model=True)
)
coxnet_pred.set_params(**gcv.best_params_)
coxnet_pred.fit(X_test, Y_test)

#surv_fns = coxnet_pred.predict_survival_function(X_test)
